# Advanced Python Module 7: Python Review

In this module, we will review the topics in numpy.

In [ ]:
# %load_ext autoreload
# %autoreload 2
import tests
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.sparse import diags
from IPython import display

## Q1: 1D Numerical simulation of the Schrödinger equation 

In the quantum mechanical model, the wave function $\psi(x) = \langle x|\psi\rangle$ are the complex-valued expansion coefficients of the state of the system in the position basis. In other words, the probability density of finding a particle at some position $x_0$ is given by $|\psi(x_0)|^2$. The wave function evolves with time (i.e. the probability of finding a particle at $[x, x+dx]$ changes over time) according the Schrödinger equation,

$$i\hbar \frac{\partial\psi}{\partial t} = H\psi$$

Where $H$ is the Hamiltonian, a term that represents the total energy of the system. If we know the initial wave function $\psi(x,t=0)$, then we can numerically evolve the Schrödinger equation for the wave function at some later time. This is precisely our task in this exercise. Specifically, we will time-evolve a wave packet traveling towards a potential barrier and observe quantum tunneling.

### Initial state

We shall choose a coherent state for our initial wave function. A coherent state is a minimum uncertainty wave packet ($\Delta x\Delta p = \hbar/2$) such as the ground state of a harmonic oscillator $|0\rangle$. In general, a coherent state with average position $\langle x\rangle =a$ and average momentum $\langle p\rangle =b$ is given by the wave function [ref](http://bohr.physics.berkeley.edu/classes/221/notes/harmosc.pdf),

$$\psi_0(x) = \frac{1}{\pi^{1/4}} \exp{\left[-\frac{(x-a)^2}{2} + ibx - \frac{iab}{2}\right]}$$

### Q1a: Initial condition

Implement the function ``psi_0`` which returns the wave function of a coherent state. ``x`` is an ndarray of positions to evaluate the wave function, ``a`` is the average position, and ``b`` is the average momentum. Your function should return an ndarray and should have *default values: ``a``=-30 and ``b``=3.*

Hint: numpy can handle complex values. For example, $i\cdot b$ = `1j*b`.

In [ ]:
# write the function psi_0
# inputs: x, a and b. a and b have default values.
# returns: the wave function for given input.

...

In [ ]:
# test your work!
tests.run('test_1a', psi0)

### The Hamiltonian

Note: for simplicity, in this exercise we will work in atomic units (A.U.) where $e=m=\hbar=1$. In atomic units, the characteristic quantities are:
- Distance: is the Bohr radius $a_0 = \hbar^2/me^2 \text{ (G.U.)} = 0.5$ angstrom
- Energy: is $K_0 = e^2/a_0 = 27$ eV
- Time: is $T_0 = \hbar^3/me^4 = 2.4\times 10^{-17}$ sec.

The Hamiltonian is given as,
$$H = \frac{p^2}{2} + V(x) = -\frac{1}{2}\frac{d^2}{dx^2} + V(x)$$

### Finite difference method

We will use the finite difference method to perform a numerical simulation of the Schrödinger equation. In this method, we first discretize the wave function. In other words, we define a range $[x_\text{min}, x_\text{max}]$ and sample $\psi_0(x)$ in increments of $\delta x$ within the range.

### Q1b: Discretizing

We have provided some default values for ``x_min``, ``x_max``, and ``delta_x``. Define the following variables:
- ``x`` is an ndarray of x values from \[``x_min``, ``x_max``\] incremented by ``delta_x``
- ``n`` is the number of samples (length of ``x``)
- ``psi`` is the initial wave function ``psi_0`` sampled at ``x``

In [ ]:
x_min = -85
x_max = 65
delta_x = 0.1

x = ...
n = ...
psi = ...

In [ ]:
# test your work!
tests.run('test_1b', x, n, psi)

The potential $V(x)$ is a simple square potential,
$$V(x) = 
\begin{cases}
V_h,&0\leq x\leq V_w\\
0,&\text{else}
\end{cases}
$$

### Q1c: The potential

You are given the values for ``v_w`` and ``v_h``. Calculate the potential ``V`` sampled at ``x``. ``V`` is an ndarray.

In [ ]:
v_w = 7 # (a_0)
v_h = 6 # (K_0)

# calculate V


In [ ]:
# test your work!
tests.run('test_1c', V)

The Hamiltonian is an operator (matrix) that acts on the vector (array) ``psi``. Recall that $H = -\frac{1}{2}\frac{d^2}{dx^2} + V(x)$. Since ``psi`` has dimension $n$, the Hamiltonian must have dimension $n\times n$.

It is straightforward to see how we can translate $V(x)$ into a matrix: $(V(x)\psi(x))_i = V(x_i)\psi(x_i)$, so $V(x)$ is a diagonal matrix with elements $V_{ii} = V(x[i])$.

The kinetic term is a bit tricker. For the finite difference method,

$$\frac{d^2}{dx^2} f(x[i]) \approx \frac{f(x[i-1])-2f(x[i])+f(x[i+1])}{\delta x^2}$$

Placing this in matrix form and combining with the potential term,

$$H = \frac{1}{2\delta_x^2}
\begin{pmatrix}
 2 & -1 & &  \\
 -1 & 2& -1 &  \\
 & -1 & 2 & -1 \\
 & & &  \ddots
\end{pmatrix} 
+
\begin{pmatrix}
  V(x[0])& & & \\
  & V(x[1]) & & \\
 &  & V(x[2]) &\\
 & &  & \ddots
\end{pmatrix} 
$$

The unfilled spaces in the matrices are just 0.

### Q1d: the Hamiltonian
Calculate the matrix ``H``. Hint: read the documentation for the ``diags`` function in ``scipy.sparse``. Play around with the function and think about what the different arguments mean. Try copying over the examples from the documentation and working with them in this notebook. Don't forget to convert the sparse matrix into an ndarray. **Feel free to add cells and visualize the output for every step you take.**

In [ ]:
# calculate H


In [ ]:
tests.run('test_1d', H)

Now, let's time evolve! Using the [Crank-Nicolson method](https://en.wikipedia.org/wiki/Finite_difference_method#Crank%E2%80%93Nicolson_method), we can discretize the Schrödinger equation as,

$$\frac{\psi(x, t+\delta t) - \psi(x, t)}{dt} \approx \frac{1}{2i}H\psi(x,t) + \frac{1}{2i}H\psi(x, t+\delta t)$$

Solving for $\psi(x, t+\delta t) $,

$$\psi(x, t+\delta t) = \underbrace{\left(1-\delta t\frac{H}{2i}\right)^{-1}\left(1+\delta t\frac{H}{2i}\right)}_\text{CNO} \psi(x,t)$$

The dot product of the "Crank-Nicholson Operator" (CNO) with the state at time $t$ yields an approximation for the state at time $t+\delta t$.

### Q1e: the Crank-Nicholson operator

Given ``delta_t``=0.25, calculate the Crank-Nicholson Operator ``CNO``. 

Hints: 
- numpy has a number of functions for performing linear algebra operations
- In particular, read the documentation for ``numpy.identity``

In [ ]:
delta_t = 0.25

#calculate CNO

In [ ]:
tests.run('test_1e', CNO)

### Animation

We're now ready to simulate! A animation of the time evolution of ``psi_0`` is provided for you. A sketch of the code:
1. ``FuncAnimation`` calls the function ``animate`` for each frame of the animation.
2. ``animate`` computes the time-evolved by $\delta_t$ wave function by taking the matrix product of the current wave function with ``CNO``
3. The animation is rendered to an html5 video which is then displayed.

### Q1f: animating!

Run the code below!

In [ ]:
fig, ax = plt.subplots()

axt = ax.twinx()
axt.plot(x, V, 'r')
axt.set_ylabel('Potential ($K_0$)')

psi = psi0(x)
wf, = ax.plot(x, abs(psi)**2)
text = ax.text(0.05, 0.95, '', horizontalalignment='left', verticalalignment='top', transform=ax.transAxes)
ax.set_xlabel('Position ($a_0$)')
ax.set_ylabel('Probability density ($a_0^{-1}$)')

print('Frame: 0', end='')
def animate(i):
    global psi
    psi = np.dot(CNO, psi)
    wf.set_ydata(abs(psi)**2)
    text.set_text(f'$t=${i*0.024:.3f} fs')
    print(f'\rFrame: {i+1}', end='', flush=True)
    return wf,

anim = animation.FuncAnimation(fig, animate, frames = 190, interval = 100)

video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()    

Notice in the simulation that there is a non-zero probability density that appears to the right of the potential barrier. We refer to this as quantum tunneling. Allow me to make some remarks.

**\[Optional remarks\]**: In quantum mechanics courses, the issue of tunneling is often treated in the case where a plane wave travels towards a potential barrier. Solving for solutions to the time-independent Schrödinger equation, the energy eigenstate has a non-zero probability density to the right of the potential. However, this example is not physical because a plane wave is not normalizable. In other words, the wave function can never actually be a plane wave. Coherent states *are* physical states (that are linear combinations of plane waves); hence, our simulation is more realistic in this respect.

However, a coherent state is not a energy eigenstate. This begs the question: what does it mean to say the particle has tunneled? In the case of a plane wave with energy eigenvalue E, if E$<V_h$ (the height of the barrier), we say that a classical particle with similar energy can not be observed on the other side of the barrier---hence the quantum particle must have tunneled through. (Although...we must be careful even with this analogy: when we measure the position of the particle, the wave function is projected onto a state that is no longer an energy eigenstate.)

It is difficult to find such an analogy for a coherent state. Perhaps we can define E, the energy of the particle in the classical limit, as $\langle p\rangle^2/2$. But we can find $\psi_0(x)$ to have larger values of momentum than $\langle p\rangle$ with relatively large probabilities, so this is not entirely convincing. Perhaps $E = p_\text{max}^2/2$ where $p_\text{max}$ is the maximum value of momentum we can measure $\psi_0(x)$. But, the momentum space wave function of a Gaussian (e.g. the coherent state with $\langle x\rangle = \langle p\rangle=0$) is Gaussian, so $p_\text{max}$ is unbounded and tunneling is not classically forbidden.

Perhaps these two arguments taken together give us the most sensible interpretation: in the classical limit, we might expect the distribution of momentum to approach $\langle p\rangle$ and $\Delta p$ to approach 0. Therefore, if $\langle p\rangle^2/2<V_h$, the probability of measuring the particle to be in a sufficiently large momentum state ($p^2/2>V_h$) approaches zero: *tunneling is clasically allowed, but highly improbable*. On the length and energy scales of an electron, tunneling is a common affair; but on human scales, the probability is non-zero, but so small as to be forbidden for all intents and purposes.

### Q1g: normalization

Time evolution is unitary (preserves normalization). In other words, we expect,
$$\int |\psi(x',t)|^2dx' = \int |\psi(x',t+\delta_t)|^2dx' = 1$$

Show that this is indeed the case. In other words, ``psi`` has now been time-evolved to the final wave function. Calculate, ``norm`` = $\delta_x\cdot \sum |\psi(x[i])|^2 $, which we expect to have value 1.


In [ ]:
norm = ...
print(f'Norm: {norm}')

In [ ]:
tests.run('test_1g', norm)

## Submission

Check to make sure that you have answered all questions. Run all the cells so that all output is visible. Finally, export this notebook as a PDF (File/Download As/PDF via LaTeX (.pdf)) and submit to bCourses.

Inspired by a similar blog-series by Patrick Gono. Created and edited by the ULAB staff. Last updated: December 2021.